In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
import os
import urllib.request

## Merge tests data with oxford database

In [ ]:
# read test data remove spaces from column names...
covid_tests = (pd.read_csv("../data_sources/tests_latest.csv", 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)
                .rename({'ISO code': 'Code'}, axis=1))
covid_tests.columns = covid_tests.columns.str.replace(' ', '_')
# drop rows with null Code
covid_tests = covid_tests[covid_tests.Code.notna()]
covid_tests.info()

In [ ]:
oxford = pd.read_csv("../data_sources/OxCGRT_latest.csv", 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)
oxford.info()

We have to merge on two levels: country code and date, so lets index the dataframes accordingly

In [ ]:
covid_tests = covid_tests.set_index(['Code', 'Date'])
covid_tests.info()

In [ ]:
oxford = oxford.set_index(['CountryCode', 'Date'])

In [ ]:
# oxford_tests =(oxford
#                .join(covid_tests.rename_axis(oxford.index.names), how='left')
#                .drop(['Entity', 'new_tests_per_thousand_7day_smoothed Annotations'], axis=1)
#                .rename({'new_tests_per_thousand_7day_smoothed': 'covid_tests'})
#               )
oxford_tests =(oxford
               .join(covid_tests.rename_axis(oxford.index.names), how='left')
#                .drop(['Entity', 'new_tests_per_thousand_7day_smoothed Annotations'], axis=1)
#                .rename({'new_tests_per_thousand_7day_smoothed': 'covid_tests'})
              )
oxford_tests

Of course this has lots of NaN values in the tests column, but we can find a way to deal with those later when we use it on a predictor. Fpr the time being, lets save it

In [ ]:
oxford_tests.to_csv("../data_sources/OxCGRT_latest_with_tests.csv")

Now pack it up in a function 

In [ ]:
def add_test_data(oxford_path, tests_path):
    covid_tests = (pd.read_csv(tests_path, 
                     parse_dates=['Date'],
                     encoding="ISO-8859-1",
                     dtype={"RegionName": str,
                            "RegionCode": str},
                     error_bad_lines=False)
                    .rename({'ISO code': 'Code'}, axis=1)
                  )
    covid_tests.columns = covid_tests.columns.str.replace(' ', '_')
    # drop rows with null Code
    covid_tests = covid_tests[covid_tests["Code"].notna()]
    covid_tests = covid_tests.set_index(['Code', 'Date'])
    oxford = pd.read_csv(oxford_path, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)
    oxford = oxford.set_index(['CountryCode', 'Date'])
    oxford_tests =(oxford
                   .join(covid_tests.rename_axis(oxford.index.names), how='left')
                  )
    return oxford_tests.reset_index()

In [ ]:
oxford_tests = add_test_data("../data_sources/OxCGRT_latest.csv", "../data_sources/tests_latest.csv")
oxford_tests

## Update Oxford and test data

Lets wrap up the workflow to update and merge the Oxford and tests datasets

In [ ]:
def update_OxCGRT_tests():
    # source of latest Oxford data
    OXFORD_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
    # source of latest test data
    TESTS_URL = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/testing/covid-testing-all-observations.csv"
    # store them locally
    OXFORD_FILE = '../data_sources/OxCGRT_latest.csv'
    TESTS_FILE = '../data_sources/tests_latest.csv'
    urllib.request.urlretrieve(OXFORD_URL, OXFORD_FILE)
    urllib.request.urlretrieve(TESTS_URL, TESTS_FILE)
    return add_test_data(OXFORD_FILE, TESTS_FILE)

In [ ]:
latest_oxford_tests = update_OxCGRT_tests()

In [ ]:
latest_oxford_tests

## Country characterization

Now we are going to mcollect some static data aboput each country to try to capture how the different NPIs might work nacroos countries.

We have the folloing datasets (gathered from [Our World in Data (OWD)](https://ourworldindata.org/)):

* Economic Freedom
* Gross per capita National Income
* Human Development Index
* Human Rights Scores
* Life Expectancy
* Mean Years of schooling
* Political Regime 

All of these datasets are timeseries with measurements for different points in time. In the simplest scenario we're going to take the most recent measure for each dataset.

In [ ]:
ef = pd.read_csv("../data_sources/economic-freedom.csv")
# Check we have the same final year for all observations
last_year = ef.Year.max()
print(len(ef.Code.unique()), (ef.groupby('Code')[['Year']].max() == last_year)['Year'].sum())

Every dataset from OWD has the same basic structure, so we can easily check if every dataset has all observetions for the last measured Year

In [ ]:
def check_all_final_year(data_path):
    df = pd.read_csv(data_path)
    last_year = df.Year.max()
    try:
        assert len(df.Code.unique()) == (df.groupby('Code')[['Year']].max() == last_year)['Year'].sum()
    except Exception as e:
        print(e)
        return False
    return True

In [ ]:
paths = ["../data_sources/economic-freedom.csv",
         "../data_sources/gross-national-income-per-capita.csv",
         "../data_sources/human-development-index.csv",
         "../data_sources/life-expectancy.csv",
         "../data_sources/mean-years-of-schooling-long-run.csv",
         "../data_sources/political-regime-updated2016-distinction-democracies-and-full-democracies.csv"]
for p in paths:
    print(p, check_all_final_year(p))

As we can see, there are some missing observations for the last year in most datasets. The simplest way to deal with those is a forward fill, propagating forward the last valid observation for each country.

It is also important to notice that the datasets contain regional and global values without countr codes so we need to clean those beforehand 

In [ ]:
df = pd.read_csv("../data_sources/human-development-index.csv")
# remove missing country codes
df = df[df['Code'].notna()]
# forward fill for each country
df['Human Development Index (UNDP)'] = (df
                                       .groupby('Code')[['Human Development Index (UNDP)']]
                                       .apply(lambda x: x.fillna(method='ffill'))
                                      )
# check that we don't have any repeated Codes
df = df[df['Year'] == df.Year.max()]
(df.groupby('Code').size() > 1).sum()

Npw lets get a DataFrame with every country in the Oxford database and all variables from OWD. Of course we're going to get lots of NAs, we'll deal with those later

In [ ]:
countries = pd.DataFrame(oxford['CountryCode'].unique(), columns=['CountryCode'])
rename_dict = {'Economic Freedom of the World': 'economic_freedom',
               'GNI per capita, PPP (constant 2011 international $)': 'gni_per_capita',
                'Human Development Index (UNDP)': 'human_development',
                'Life expectancy': 'life_expectancy',
                'Average Total Years of Schooling for Adult Population (Lee-Lee (2016), Barro-Lee (2018) and UNDP (2018))': 'average_years_school',
                'Political Regime (OWID based on Polity IV and Wimmer & Min)': 'political_regime'
}
data_columns = {"../data_sources/economic-freedom.csv": 'Economic Freedom of the World',
                 "../data_sources/gross-national-income-per-capita.csv": 'GNI per capita, PPP (constant 2011 international $)',
                 "../data_sources/human-development-index.csv": 'Human Development Index (UNDP)',
                 "../data_sources/life-expectancy.csv": 'Life expectancy',
                 "../data_sources/mean-years-of-schooling-long-run.csv": 'Average Total Years of Schooling for Adult Population (Lee-Lee (2016), Barro-Lee (2018) and UNDP (2018))',
                 "../data_sources/political-regime-updated2016-distinction-democracies-and-full-democracies.csv": 'Political Regime (OWID based on Polity IV and Wimmer & Min)'    
}

for p in paths:
    df = pd.read_csv(p)
    df = df[df['Code'].notna()]
    df[data_columns[p]] = (df
                                .groupby('Code')[[data_columns[p]]]
                                .apply(lambda x: x.fillna(method='ffill'))
                                )
    df = df[df['Year'] == df.Year.max()]
    countries = countries.merge(df, left_on='CountryCode', right_on='Code', how='left')
    
countries = countries.rename(rename_dict, axis=1)[['CountryCode'] + list(rename_dict.values())]
countries

Lets save this dataset 

In [ ]:
countries.to_csv("../data_sources/owd_by_country.csv")